# 파일 검색 및 검색 (File Search and Retrieval)

- **OpenAI 방식의 RAG 구현**

| 구분           | File Search                                              | Retrieval                                            |
| ------------ | -------------------------------------------------------- | ---------------------------------------------------- |
| **핵심 목적**    | 업로드된 파일을 빠르게 찾아서 모델이 참고하도록 돕는 도구                       | 임베딩 기반으로 문서 의미를 이해하고, 가장 관련된 내용을 뽑아주는 시스템          |
| **디자인 철학**   | 도구(Tool) — Responses API 내부에서 모델이 즉시 쓸 수 있도록 설계됨    | 서비스 레벨 API — 임베딩 + 검색을 조합해서 사용자가 직접 구축하는 검색 시스템 |
| **사용 대상**    | Responses API에서 `tools=[{"type": "file_search"}]` 형태로 지정 | 독립적인 `client.vector_stores.search()` 호출로 개발자가 직접 사용  |
| **내부 구조**    | 모델이 도구로 호출 → 백엔드에서 Vector Store 검색 수행                    | 사용자가 직접 Vector Store 생성, 파일 업로드, 검색, 필터링 수행          |
| **통합성**      | 모델 대화형 통합에 최적화 (자동 grounding, citations 등)               | 검색 로직을 직접 제어 가능 (chunk 크기, score threshold, 필터 등)    |
| **유스케이스 예시** | 이 PDF에서 환불 정책 찾아줘 → File Search로 모델이 자동 호출             | 내 FAQ 데이터에서 ‘환불’ 관련 문장 의미 검색 → Retrieval로 직접 쿼리    |

### 용도
단순 모델 대화에서 문서 참조만 원하면 `file_search`, 검색 시스템 구축이면 `retrieval` 사용

### 벡터 스토어 (Vector stores)

Vector store(벡터 스토어) 는 Retrieval API 와 File Search 도구에서 의미 기반 검색(Semantic Search) 을 가능하게 하는 핵심 컨테이너입니다. 
파일을 벡터 스토어에 추가하면, 해당 파일은 자동으로 청크 단위로 분할(chunked) 되고, 임베딩 (embedding) 이 생성되며, 인덱싱(indexing) 되어 검색 가능한 형태로 저장됩니다.  

- 구조 및 구성 요소
    - 벡터 스토어는 내부적으로 vector_store_file 객체들을 포함합니다.
    - 이 객체들은 실제로는 file 객체를 기반으로 만들어진 래퍼(wrapper) 타입입니다.
    - 즉, file → 실제 업로드된 원본 파일, vector_store.file → 해당 파일을 임베딩/인덱싱하여 벡터 스토어에 등록한 형태

| 객체 유형 (Object type)   | 설명 (Description)                                                                                                                                                  |
| --------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **file**              | Files API를 통해 업로드된 콘텐츠를 나타냅니다.                                                              |
| **vector_store**      | 검색 가능한 파일들을 담는 **컨테이너(Container)** 입니다.                                                                                                                           |
| **vector_store.file** | 벡터 스토어에 연결된 개별 파일을 나타내는 **래퍼(wrapper) 타입**입니다.<br>해당 파일은 이미 **청크 분할**, **임베딩**, **인덱싱** 처리가 완료된 상태입니다.<br>또한, **속성(attribute) 기반 필터링**을 위한 `attributes` 맵을 포함합니다. |

- 가격
| **저장 용량 (Storage)**      | **요금 (Cost)**      |
| ------------------------ | ------------------ |
| **총 1GB 이하 (모든 스토어 합산)** | **무료 (Free)**      |
| **1GB 초과분**              | **하루당 $0.10 / GB** |


## 파일 검색 (File Search)
모델이 응답을 생성하기 전에 관련 정보를 파일에서 검색할 수 있도록 합니다.
파일 검색은 Responses API에서 사용할 수 있는 도구입니다. 모델이 시맨틱 및 키워드 검색을 통해 이전에 업로드된 파일의 지식 기반에서 정보를 검색할 수 있도록 합니다. 벡터 저장소를 생성하고 파일을 업로드하면, 모델이 이러한 지식 기반 또는 벡터 저장소에 접근할 수 있도록 하여 모델의 내재된 지식을 강화할 수 있습니다.  

OpenAI에서 관리하는 호스팅 도구이므로 실행을 처리하기 위해 직접 코드를 구현할 필요가 없습니다. 모델이 이 도구를 사용하기로 결정하면 자동으로 도구를 호출하고 파일에서 정보를 검색하여 출력을 반환합니다.

Responses API로 파일 검색을 사용하려면 먼저 벡터 저장소에 지식 기반을 설정하고 파일을 업로드해야 합니다.  

지식 기반을 설정한 후에는 모델에서 사용할 수 있는 도구 목록에 file_search 도구를 포함할 수 있으며, 검색할 벡터 저장소 목록도 포함할 수 있습니다.

In [1]:
from dotenv import load_dotenv
load_dotenv() 

True

In [2]:
from openai import OpenAI
client = OpenAI()

Model = "gpt-5-mini"

1. 벡터 저장소를 만들고 파일을 업로드합니다.

- `upload_and_poll()` - OpenAI의 Vector Store API 안에서 “파일 업로드(upload)” + “인덱싱 완료(poll)” 를 한 번에 처리하는 편의 함수

In [4]:
vector_store = client.vector_stores.create(        # vector store 생성
    name="Support FAQ",
)

client.vector_stores.files.upload_and_poll(        # file 업로드
    vector_store_id=vector_store.id,
    file=open("data/네이버.pdf", "rb")
)

VectorStoreFile(id='file-7cdDqL5bqQewVaec6584K5', created_at=1762565082, last_error=None, object='vector_store.file', status='completed', usage_bytes=655110, vector_store_id='vs_690e9bd70f708191807f0cd03d458ddb', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

2. 검색어를 보내 관련 결과를 얻습니다.

In [7]:
resp = client.responses.create(
    model=Model,  
    instructions="제공된 출처를 바탕으로 질의에 대한 간결한 답변을 작성하세요.",
    input="네이버의 목표주가는 얼마인가요?",
    # file_search 도구 설정
    tools=[{
        "type": "file_search",          # 파일(문서) 기반 검색 도구
        "vector_store_ids": [vector_store.id]  # 검색할 벡터 스토어 ID 목록
    }]
)

# 모델이 생성한 최종 텍스트 출력
print(resp.output_text)

한화투자증권 리서치(공표일 2024-02-05)는 네이버의 목표주가를 280,000원(투자의견: BUY)으로 제시했습니다 . 같은 리포트의 SOTP 산출표에서는 적정주가 약 279,947원(목표주가 280,000원)으로 표기되어 있습니다 .

다른 증권사들의 목표주가도 원하시면 알려드릴게요.


- 응답에 검색 결과 포함
  
출력 텍스트에서 주석(파일 참조)을 볼 수 있지만, 파일 검색 호출은 기본적으로 검색 결과를 반환하지 않습니다.  
응답에 검색 결과를 포함하려면 응답을 생성할 때 include 매개변수를 사용할 수 있습니다.  

In [8]:
resp = client.responses.create(
    model=Model, 
    instructions="제공된 출처를 바탕으로 질의에 대한 간결한 답변을 작성하세요.",
    input="네이버의 목표주가는 얼마인가요?",

    # file_search 도구 설정
    tools=[{
        "type": "file_search",               # 파일 기반 검색 도구 사용
        "vector_store_ids": [vector_store.id]  # 검색할 벡터 스토어 ID (사전에 생성한 벡터 스토어)
    }],

    # include 옵션: 응답에 검색된 파일 결과(file_search_call.results)를 포함시킴
    include=["file_search_call.results"]
)

# 모델이 생성한 요약/최종 답변 출력
print(resp.output_text)

한화투자증권 리서치(공표일 2024-02-05)는 네이버(NAVER, 035420)의 목표주가를 280,000원으로 제시했고(투자의견: Buy) 보고서 기준 현재주가는 221,500원으로 상향여력은 약 26.4%로 기재되어 있습니다 .



In [13]:
# resp.output

### 다수 파일의 vector store 생성

In [15]:
# "Financial Statements"라는 벡터 스토어 생성
vector_store = client.vector_stores.create(name="Financial Statements")
 
# OpenAI에 업로드할 파일 준비
file_paths = ["data/goog-10k.pdf", "data/brka-10k.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# 업로드 및 폴링 SDK 도우미를 사용하여 파일을 업로드하고 벡터 스토어에 추가,
# 파일 배치의 완료 상태를 폴링합니다.
file_batch = client.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# 이 작업의 결과를 보기 위해 상태 및 파일 개수를 출력할 수 있습니다.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


In [18]:
response = client.responses.create(
    model=Model,
    input=[
        {"role": "user", "content": "업로드한 자료에 있는 Google의 당기 순이익은 얼마였나요? BERKSHIRE HATHAWAY INC. 와 어느 쪽의 당기 순이익이 더 많았나요?"}
    ],
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }]
)

print(response.output_text)

업로드하신 자료에 따른 답변입니다.

- Alphabet(구 Google) 문서에는 2016회계연도 순이익(당기순이익)이 19.5 billion 달러(=$19.5B)라고 기재되어 있습니다 .  
- Berkshire Hathaway Inc. 문서에는 2020회계연도 전체 순이익(net earnings)이 $43,253 million(≈$43.253 billion)이며, Berkshire Hathaway 주주 귀속 순이익은 $42,521 million이라고 기재되어 있습니다 .

따라서, 문서에 기재된 수치끼리 단순 비교하면 Berkshire Hathaway 쪽의 순이익이 더 큽니다. (단, 위의 Google 수치는 2016년 수치이고 Berkshire 수치는 2020년 수치이므로 같은 연도의 비교를 원하시면 말씀해 주세요.)


In [19]:
# 새 벡터 스토어 생성
vector_store = client.vector_stores.create(name="기업분석보고서")
 
# OpenAI에 업로드할 파일 준비
file_paths = ["data/네이버.pdf", "data/LG엔솔.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# 업로드 및 폴링 SDK 도우미를 사용하여 파일을 업로드하고 벡터 스토어에 추가,
# 파일 배치의 완료 상태를 폴링합니다.
file_batch = client.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# 이 작업의 결과를 보기 위해 상태 및 파일 개수를 출력할 수 있습니다.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


In [20]:
response = client.responses.create(
    model=Model,
    input=[
        {"role": "user", "content": "LG에너지솔루션의 2024년 하반기 전망을 요약해 주세요. 에코프로비엠은 매출액이 줄은 것에 비해 왜 영업이익이 더 크게 줄어들었나요?"},
        {"role": "user", "content": "2023년 4분기의 네이버의 영업이익은 얼마였나요? 한화 투자 증권이 목표가를 상향 조정한 이유는 무엇인가요?"}
    ],
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }]
)

print(response.output_text)

아래는 요청하신 내용 정리입니다.

1) LG에너지솔루션 — 2024년 하반기 전망(요약)
- 요약: 2Q24 실적이 컨센서스에 못 미칠 것으로 전망되며(수요 둔화·ESS 적자 영향), 하반기 펀더멘털은 주요 고객사들의 신차 출시(특히 GM)의 판매 회복 여부에 달려 있다는 점이 핵심입니다. GM의 판매 성장세가 다른 고객사의 부진을 얼마나 상쇄하느냐가 관건이라는 평가입니다. 이에 따라 목표주가가 일부 하향(약 8% 하향)되었고 투자심리는 신차·고성장 고객사 가시성 회복 시 개선될 것으로 보입니다. 또한 사업부별로는 자동차전지와 ESS 부문에서 수익성 약화가 예상됩니다.  
- 주요 포인트
  - 2Q24 컨센서스 하회: 선도 전기차업체향 수요 부진 + ESS 적자 영향으로 실적이 약화될 전망. 
  - 고객 포트폴리오별 차별화: Tesla·VW·Volvo·르노·Ford 등은 판매 부진, GM만 신차로 성장세. GM의 판매 궤적이 하반기 실적·주가에 핵심 변수. 
  - 제품 믹스·ASP 이슈: 일부 고객의 LFP 트림 확대로 ASP 하락 및 마진 압박 발생. 
  - 세부실적(리포트 추정): 2Q 매출/영업이익 등 부문별 전망 및 ESS·자동차전지의 영업이익률 악화 등이 보고서에 제시되어 있음. 

2) 에코프로비엠(에코프로비엠) — “매출 감소보다 영업이익이 더 크게 줄어든 이유”에 대해
- 현재 업로드된 파일들(제공하신 문서)에서는 에코프로비엠 관련 리포트·수치가 확인되지 않습니다. 관련 문서를 올려주시면 해당 파일을 보고 정확한 원인(제품 믹스, 원재료비 상승, 판관비 증가, 일회성 비용·평가손실, 재고평가손실 등 중 실제 원인)을 근거로 분석해 드리겠습니다. (현재 문서 검색에서는 에코프로비엠 자료가 없음)
- 참고로 일반적으로 “매출 감소 대비 영업이익이 더 크게 감소”하는 원인은 대개 다음 중 하나(또는 복합작용)입니다:
  - 원가(원재료) 상승이나 ASP(판매단가) 하락으로 매출총이익률이 급격히 악화된 경우
  - 제품 믹스 변화(고마진 품목 

# RAG (Retrieval Augmented Generation) - 검색 증강 생성


### 시맨틱 검색 (Semantic Search)

시맨틱 검색(Semantic Search) 은 벡터 임베딩(Vector Embeddings) 을 활용하여 의미적으로 연관된 결과를 찾아내는 검색 기법입니다.
이 방식의 핵심은, 단순히 키워드가 일치하지 않아도 의미가 유사한 결과를 찾을 수 있다는 것입니다.
즉, 기존의 키워드 기반 검색으로는 놓칠 수 있는 결과를 포착할 수 있습니다.

- 예시
검색 문장: “우리가 달에 간 게 언제였지?”  
가능한 결과 예시는 다음과 같습니다.
| 문장(텍스트)                    | 키워드 유사도 | 의미 유사도 |
| -------------------------- | ------- | ------ |
| 첫 번째 달 착륙은 1969년 7월에 일어났다. | 0%      | 65%    |
| 달에 간 첫 번째 사람은 닐 암스트롱이었다.   | 27%     | 43%    |
| 내가 달 케이크를 먹었을 때, 정말 맛있었다.  | 40%     | 28%    |


Retrieval API는 데이터에 대해 의미 기반 검색(Semantic Search) 을 수행할 수 있게 해줍니다.
이는 단순히 키워드가 일치하는 결과가 아니라, 의미적으로 유사한 정보를 찾아주는 기술입니다.  
예를 들어, “환불 정책이 뭐야?”라고 물으면 “반품 가능 기간은 30일입니다” 같은 문장을 찾아줍니다.  
이 검색 기능은 단독으로도 유용하지만, 모델과 결합하면 매우 강력한 지식 기반 응답 생성 시스템을 만들 수 있습니다.  

Retrieval은 벡터 스토어(Vector Store) 라는 데이터 인덱스를 기반으로 작동합니다.  
문서를 업로드하면 내부적으로 임베딩(embedding) 을 생성하고, 이를 통해 의미적으로 유사한 문서를 빠르게 찾아낼 수 있습니다.

### 벡터 스토어 생성/파일 업로드

In [21]:
vector_store = client.vector_stores.create(        # vector store 생성
    name="Support FAQ",
)

client.vector_stores.files.upload_and_poll(        # file 업로드
    vector_store_id=vector_store.id,
    file=open("data/customer_policy.txt", "rb")
)

VectorStoreFile(id='file-AAsk3jn9omvv5K1p4qwkka', created_at=1762569254, last_error=None, object='vector_store.file', status='completed', usage_bytes=55876, vector_store_id='vs_690eac242bc881919fae1fa933df4c8c', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [22]:
# 검색 쿼리를 전송하여 관련 결과를 가져옵니다.
user_query = "서비스가 중단될 경우 영업상 발생한 손해는 어떻게 처리되나요?"

results = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query=user_query,
)

In [24]:
# file_search 결과에서 텍스트(content.text)만 추출하여 하나의 문자열로 합치는 코드

combined_text = "\n\n---------------\n\n".join(   # 각 문서 간 구분선을 추가하여 보기 쉽게 출력
    c.text                                    # 각 검색 결과(content)의 텍스트 속성(text)을 가져옴
    for r in results.data                     # file_search 결과의 data 리스트를 순회
    for c in r.content                        # 각 data 항목 안의 content 리스트를 순회
    if hasattr(c, "text")                     # content 객체가 text 속성을 가지고 있을 때만 선택
)

print(combined_text)

변경. 당사는 수시로 가격을 변경할 수 있습니다. 구독료를 인상할 경우 최소한 30일 전에 사전 통지하며 가격 인상은 귀하의 다음번 갱신부터 효력이 발생되므로 가격인상에 동의하지 않을 경우 구독을 취소할 수 있습니다.

해지 및 중단
해지. 귀하는 언제든지 본 서비스 이용을 중단할 수 있습니다. 당사는 다음과 같이 판단할 경우 귀하의 본 서비스에 대한 이용을 중단하거나 해지 또는 귀하의 계정을 삭제할 권리를 보유합니다:

본 약관 또는 당사의 이용 정책⁠을 위반한 경우.
법률 준수를 위하여 위와 같이 조치하여야 할 경우.
귀하의 본 서비스 이용이 OpenAI, 당사 사용자 또는 타인에게 위험이나 피해를 초래할 수 있을 경우.
귀하의 계정이 1년 이상 휴면 상태이고 유료계정이 없을 경우 귀하의 계정을 해지할 수 있습니다. 위와 같은 경우 사전 통지합니다.

이의 신청. 당사가 귀하의 계정을 실수로 정지하거나 해지하였다고 판단한 경우, 당사 지원팀에 연락⁠(새 창에서 열기)하여 이의를 제기할 수 있습니다.

서비스의 중단
당사는 본 서비스를 중단하기로 결정할 수 있으나, 중단하는 경우, 사전 통지할 것이며 미사용 선불 서비스는 환불됩니다.

보증의 부인
본 서비스는 “있는 그대로” 제공됩니다. 법률상 금지되는 범위를 제외하고, 당사 및 당사의 계열사 및 라이선스 제공자는 본 서비스와 관련하여 어떠한 보장(명시적, 묵시적, 법적 또는 기타)도 하지 않으며, 상업성, 특정 목적 적합성, 만족스러운 품질, 비침해 및 향유에 관한 보장과 거래 과정에서 또는 거래상 이용 과정에서 발생하는 모든 보장을 부인합니다. 당사는 본 서비스가 중단 없이, 정확하고 오류가 없으며, 또는 콘텐츠가 안전하거나 손실 또는 변경되지 않을 것임을 보장하지 않습니다.

---------------

서비스의 중단
당사는 본 서비스를 중단하기로 결정할 수 있으나, 중단하는 경우, 사전 통지할 것이며 미사용 선불 서비스는 환불됩니다.

보증의 부인
본 서비스는 “있는 그대로” 제공됩니다. 법률상 금

### 응답 합성 (Synthesizing responses)
검색 쿼리를 수행한 후에는 그 결과를 바탕으로 응답을 종합(synthesize) 하고 싶을 때가 있습니다.  
이를 위해 OpenAI의 모델을 활용할 수 있으며, 검색 결과(results) 와 원래의 질의(original query) 를 함께 제공하면,  
모델이 해당 자료에 근거한 사실 기반의 응답(grounded response) 을 생성해 줍니다.

In [25]:
response = client.responses.create(
    model=Model,
    instructions="제공된 출처를 바탕으로 질의에 대한 간결한 답변을 작성하세요.",
    input=f"Sources: {combined_text}\n\nQuery: '{user_query}'"
)

print(response.output_text)

간단히 정리하면 다음과 같습니다.

- 서비스 중단 시(회사가 서비스를 완전히 중단하기로 결정한 경우) 회사는 사전 통지 후 미사용 선불 서비스는 환불해 줍니다.  
- 다만 회사는 본 서비스를 “있는 그대로” 제공하며, 영업 손실(이윤·영업권·이용 손실 등 포함)의 배상에 대해 간접·결과적·특별·징벌적 손해에 대해 책임을 지지 않는다고 명시하고 있습니다. 즉, 서비스 중단으로 발생한 영업상 손해(매출 손실·이익 손실 등)는 원칙적으로 회사가 배상하지 않습니다.  
- 회사의 총 책임 한도는 해당 책임 발생 전 12개월 동안 귀하가 지급한 금액과 $100 중 더 큰 금액으로 제한됩니다(단, 일부 관할구역에서는 특정 보증 부인이나 손해배상 제한이 허용되지 않아 달리 적용될 수 있음).  
- 결론적으로 미사용 선불금 환불을 제외하면, 통상적인 영업상 손해에 대한 배상은 기대하기 어렵고, 관할법에 따라 달라질 수 있으므로 필요시 지원팀에 문의하거나(정지·해지 이의 제기 등) 법적 조언을 받으시기 바랍니다.
